In [1]:
! pip install numpy sympy gmpy2 matplotlib

In [2]:
class FiniteField:
    def __init__(self, value, prime):
        self.value = value % prime
        self.prime = prime

    def __add__(self, other: "FiniteField") -> "FiniteField":
        return FiniteField(self.value + other.value, self.prime)

    def __mul__(self, other: "FiniteField") -> "FiniteField":
        return FiniteField(self.value * other.value, self.prime)
    
    def __neg__(self) -> "FiniteField":
        return FiniteField(-self.value, self.prime)
    
    def __truediv__(self, other: "FiniteField") -> "FiniteField":
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime)
    
    def __repr__(self):
        return f'{self.value} (mod {self.prime})'
        
        

In [3]:
prime = 7
finite_field_list = [ FiniteField(i, prime) for i in range(7)]
finite_field_list

[0 (mod 7), 1 (mod 7), 2 (mod 7), 3 (mod 7), 4 (mod 7), 5 (mod 7), 6 (mod 7)]

In [4]:
target = FiniteField(1, prime)
target + finite_field_list[1]

2 (mod 7)

In [5]:
class Polynomial:
    def __init__(self, coefficients):
        self.coefficients = coefficients  # 리스트 형태

    def evaluate(self, x):
        return sum([coef * x**i for i, coef in enumerate(self.coefficients)])

    def __add__(self, other: "Polynomial") -> "Polynomial":
        placeholder = [0] * max(len(self.coefficients), len(other.coefficients))
        for i, coeff in enumerate(self.coefficients):
            placeholder[i] += coeff
        for i, coeff in enumerate(other.coefficients):
            placeholder[i] += coeff
        return Polynomial(placeholder)

    def __mul__(self, other: "Polynomial") -> "Polynomial":
        placeholder = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for i, self_coeff in enumerate(self.coefficients):
            for j, other_coeff in enumerate(other.coefficients):
                placeholder[i+j] += self_coeff * other_coeff
        return Polynomial(placeholder)

    def __repr__(self):
        return " + ".join([f"{coef}x^{i}" for i, coef in enumerate(self.coefficients)])

In [6]:
import numpy as np

def fft(poly, omega, n):
    # n은 평가할 점의 개수
    if n == 1:
        return poly
    even = fft(poly[0::2], omega ** 2, n // 2)
    odd = fft(poly[1::2], omega ** 2, n // 2)
    factor = [omega ** i for i in range(n // 2)]
    return [even[i] + factor[i] * odd[i] for i in range(n // 2)] + \
           [even[i] - factor[i] * odd[i] for i in range(n // 2)]

In [7]:
def vanishing_polynomial(omega, n):
    result = Polynomial([1])  # 상수항 1
    for i in range(n):
        result *= Polynomial([-omega**i, 1])  # (X - \omega^i)
    return result

In [8]:
def permutation_check(f_vals, g_vals):
    product = 1
    for f, g in zip(f_vals, g_vals):
        product *= f / g
    return product == 1

In [9]:
# Polynomial (x1+x2)(x2+w)
def gate0(v1: FiniteField, v2: FiniteField) -> FiniteField:
    return v1 + v2

def gate1(v2: FiniteField, w: FiniteField) -> FiniteField:
    return v2 + w

def gate2(gate0_result: FiniteField, gate1_result: FiniteField) -> FiniteField:
    return gate0_result * gate1_result


In [10]:
import sympy as sp

def find_generator(prime):
    prime_factors = sp.primefactors(prime - 1)  # p-1의 소인수분해 
    for g in range(2, prime):  # 2부터 prime-1까지 제너레이터 후보 검사
        if all(pow(g, (prime - 1) // q, prime) != 1 for q in prime_factors):  # 모든 조건 확인
            return g
    return None  # 제너레이터가 없으면 None 반환

# 실행
generator = find_generator(101)
print(f"Generator for 101: {generator}")

Generator for 101: 2


In [35]:
def find_nth_root_of_unity(prime, n):
    # 제너레이터 찾기
    generator = find_generator(prime)
    # n-th root of unity 계산
    omega = pow(generator, (prime - 1) // n, prime)
    return omega

# 실행
prime = 101
n = sp.primefactors(prime - 1)[1]
omega = find_nth_root_of_unity(prime, 25)
print(f"{n}-th Root of Unity for {prime}: {omega}")

5-th Root of Unity for 101: 16


In [109]:
prime = 101
x1 = FiniteField(5, prime)
x2 = FiniteField(6, prime)
w = FiniteField(1, prime)
C = 3 # number of gates
I = 3 # number of input wires
D = 12 # degree of the polynomial
poly_T_pv = [
    (pow(omega, -3, prime), w),
    (pow(omega, -2, prime), x2),
    (pow(omega, -1, prime), x1),
    (pow(omega, 0, prime), x1),
    (pow(omega, 1, prime), x2),
    (pow(omega, 2, prime), gate0(x1, x2)),
    (pow(omega, 3, prime), x2),
    (pow(omega, 4, prime), w),
    (pow(omega, 5, prime), gate1(x2, w)),
    (pow(omega, 6, prime), gate0(x1, x2)),
    (pow(omega, 7, prime), gate1(x2, w)),
    (pow(omega, 8, prime), gate2(gate0(x1, x2), gate1(x2, w)))
] 

In [110]:
poly_T_pv

[(92, 1 (mod 101)),
 (58, 6 (mod 101)),
 (19, 5 (mod 101)),
 (1, 5 (mod 101)),
 (16, 6 (mod 101)),
 (54, 11 (mod 101)),
 (56, 6 (mod 101)),
 (88, 1 (mod 101)),
 (95, 7 (mod 101)),
 (5, 11 (mod 101)),
 (80, 7 (mod 101)),
 (68, 77 (mod 101))]

In [111]:
def lagrange_interpolation(inputs, outputs, prime):
    x = symbols('x')  # 다항식 변수
    n = len(inputs)
    poly = 0  # 초기화
    
    for j in range(n):
        # 라그랑주 다항식 구성
        lj = 1
        for i in range(n):
            if i != j:
                lj *= (x - inputs[i]) * pow(inputs[j] - inputs[i], -1, prime)
        poly += outputs[j] * lj

    # 다항식 모듈로 p에서 계산
    poly = expand(poly) % prime
    return poly

In [117]:
poly_T_inputs = [poly_T_pv[i][0] for i in range(len(poly_T_pv))]
poly_T_outputs = [poly_T_pv[i][1].value for i in range(len(poly_T_pv))]
poly_T = lagrange_interpolation(poly_T_inputs, poly_T_outputs, prime)
poly_T

Mod(434651999820320190192*x**11 - 252844496382055482569472*x**10 + 64313245847799577910738016*x**9 - 9382057058899747503374118528*x**8 + 864895310579079487887679524720*x**7 - 52300377868034879918969157099648*x**6 + 2082473448514205372873345010540864*x**5 - 53269747297989750485451264041190912*x**4 + 824653174574788171690030422915806208*x**3 - 6869934674332083354340771521484554240*x**2 + 24248293536860353042441881848999116800*x + 93, 101)

In [149]:
# Prove (1)
prime = 101
inputs = [pow(omega, i, prime) for i in [-3, -2, -1]]
outputs = [w.value, x2.value, x1.value]
poly_v = lagrange_interpolation(inputs, outputs, prime)
poly_v_values = [poly_v.evalf(subs={x: pow(omega, -i, prime)}) % prime for i in [3, 2, 1]]
poly_T_values = [poly_T.evalf(subs={x: pow(omega, -i, prime)}) % prime for i in [3, 2, 1]]
comparison = [poly_v_values[i] -  poly_T_values[i] for i in range(3)]
print(comparison)


[0, 0, 0]


In [ ]:
# Prove (2)
gates_y = [omega ** (3*i) for i in range(3)]
poly_T = lagrange_interpolation(poly_T_inputs, poly_T_outputs, prime)
poly_S = lagrange_interpolation(gates_y, [1, 1, 0], prime)

# 검증 함수 정의
def verification(i):
    left_input = poly_T.subs(x, i)
    right_input = poly_T.subs(x, omega * i)
    s_value = poly_S.subs(x, i)
    
    result = s_value * (left_input + right_input) + (1 - s_value) * (left_input * right_input) - poly_T.subs(x, omega ** 2 * i)
    return result % prime

print([verification(i) for i in gates_y])

[0, 0, 0]


In [152]:
# Prove (3)


[5, 6, 11]

In [135]:
print([comparison(i) for i in gates_y])

[59.0000000000000, 37.0000000000000, -35.0000000000000]


In [160]:
poly_T.subs(x,77) % prime

94

In [164]:
verification_point = pow(omega, 3 * 3 - 1, prime)
value_at_verification = poly_T.subs(x, verification_point) % prime

In [165]:
value_at_verification

77